Load coffea output files

In [ ]:
from coffea import hist, util
import numpy as np
import uproot

from ttgamma.utils.plotting import RebinHist, SetRangeHist

In [ ]:
nJets = 4

outputMC = util.load(f'Outputs/outputMCOther_ttgamma_condorFull_{nJets}jet.coffea')
outputMC.add(util.load(f'Outputs/outputMCSingletop_ttgamma_condorFull_{nJets}jet.coffea'))
outputMC.add(util.load(f'Outputs/outputMCTTbar1l_ttgamma_condorFull_{nJets}jet.coffea'))
outputMC.add(util.load(f'Outputs/outputMCTTbar2l_ttgamma_condorFull_{nJets}jet.coffea'))
outputMC.add(util.load(f'Outputs/outputMCTTGamma_ttgamma_condorFull_{nJets}jet.coffea'))
outputMC.add(util.load(f'Outputs/outputMCWJets_ttgamma_condorFull_{nJets}jet.coffea'))
outputMC.add(util.load(f'Outputs/outputMCZJets_ttgamma_condorFull_{nJets}jet.coffea'))

outputData = util.load(f'Outputs/outputData_ttgamma_condorFull_{nJets}jet.coffea')

Get M3 distributions, grouped into top and non-top categories, saving to a root file

In [ ]:
groupingTop = {'TopPair': ['TTGamma_Dilepton','TTGamma_SingleLept','TTGamma_Hadronic',
                           'TTbarPowheg_Dilepton', 'TTbarPowheg_Semilept', 'TTbarPowheg_Hadronic'],
               'NonTop' : ['W1jets', 'W2jets', 'W3jets', 'W4jets',
                           'DYjetsM10to50', 'DYjetsM50'
                           'ST_s_channel', 'ST_tW_channel', 'ST_tbarW_channel', 'ST_tbar_channel', 'ST_t_channel',
                           'WGamma_01J_5f',
                           'ZGamma_01J_5f_lowMass',
                           'TTWtoLNu','TTWtoQQ','TTZtoLL',
                           'GJets_HT40To100', 'GJets_HT100To200', 'GJets_HT200To400', 'GJets_HT400To600', 'GJets_HT600ToInf', 
#                           'QCD_Pt20to30_Ele', 'QCD_Pt30to50_Ele', 'QCD_Pt50to80_Ele', 'QCD_Pt80to120_Ele', 'QCD_Pt120to170_Ele', 'QCD_Pt170to300_Ele', 'QCD_Pt300toInf_Ele', 'QCD_Pt20to30_Mu', 'QCD_Pt30to50_Mu', 'QCD_Pt50to80_Mu', 'QCD_Pt80to120_Mu', 'QCD_Pt120to170_Mu', 'QCD_Pt170to300_Mu', 'QCD_Pt300to470_Mu', 'QCD_Pt470to600_Mu', 'QCD_Pt600to800_Mu', 'QCD_Pt800to1000_Mu', 'QCD_Pt1000toInf_Mu'
                          ],
              }

h = outputMC['M3'].sum('lepFlavor')
h = h.group('dataset',hist.Cat(r'dataset',r'Samples',sorting='placement'),groupingTop)
h = h.sum('category')
h = RebinHist(h,"M3",5)
h = SetRangeHist(h,"M3",50,550)

hData = outputData['M3'].sum('lepFlavor')
hData = hData.sum('dataset')
hData = hData.sum('category')
hData = hData.sum('systematic')
hData = RebinHist(hData,"M3",5)
hData = SetRangeHist(hData,"M3",50,550)
        
outputFile = uproot.recreate("RootFiles/M3_Output.root")

outputFile['dataObs'] = hist.export1d(hData)

datasets = h.axis('dataset').identifiers()
systematics = h.axis('systematic').identifiers()
for _dataset in datasets:
    for _systematic in systematics:
        outputFile[f'{_dataset}_{_systematic}'] = hist.export1d(h.integrate('dataset',_dataset).integrate('systematic',_systematic))

outputFile.close()

Get photon charged hadron isolation distributions, grouped into isolated and nonprompt categories, saving to a root file

In [ ]:
groupingPho= {"Isolated": slice(1,3),
              "NonPrompt":slice(3,5),
             }

bins = np.array([0,1.141,2.5,5,10,15,20])

h = outputMC['photon_chIso'].sum('lepFlavor')
h = h.group('category',hist.Cat(r'category',r'Samples',sorting='placement'),groupingPho)
h = h.sum('dataset')
h = h.rebin("chIso",hist.Bin("chIso",h.axis("chIso").label,bins))

hData = outputData['photon_chIso'].sum('lepFlavor')
hData = hData.sum('dataset')
hData = hData.sum('category')
hData = hData.sum('systematic')
hData = hData.rebin("chIso",hist.Bin("chIso",hData.axis("chIso").label,bins))

outputFile = uproot.recreate("RootFiles/Isolation_Output.root")
outputFile['dataObs'] = hist.export1d(hData)

categories = h.axis('category').identifiers()
systematics = h.axis('systematic').identifiers()
for _category in categories:
    for _systematic in systematics:
        outputFile[f'{_category}_{_systematic}'] = hist.export1d(h.integrate('category',_category).integrate('systematic',_systematic))
    
outputFile.close()

Save distributions of e-gamma mass

In [ ]:

groupingDataset = {'WGamma' : ['WGamma_01J_5f'],
                   "ZGamma" : ['ZGamma_01J_5f_lowMass'],
                   "Other"  : ['TTGamma_Dilepton','TTGamma_SingleLept','TTGamma_Hadronic',
                               'TTbarPowheg_Dilepton', 'TTbarPowheg_Semilept', 'TTbarPowheg_Hadronic',
                               'W1jets', 'W2jets', 'W3jets', 'W4jets',
                               'DYjetsM50', 'DYjetsM10to50',
                               'ST_s_channel', 'ST_tW_channel', 'ST_tbarW_channel', 'ST_tbar_channel', 'ST_t_channel',
                               'TTWtoLNu','TTWtoQQ','TTZtoLL',
                              ],
                  }

groupingPho= {"Genuine":slice(1,2),
              "MisIDele": slice(2,3),
              "NonPrompt":slice(3,5),
             }


h = outputMC['photon_lepton_mass_3j0t']
h = h.group('category',hist.Cat(r'category',r'Samples',sorting='placement'),groupingPho)
h = h.group('dataset',hist.Cat(r'dataset',r'Samples',sorting='placement'),groupingDataset)
h = RebinHist(h,"mass",5)
h = SetRangeHist(h,"mass",40,200)

hData = outputData['photon_lepton_mass_3j0t']
hData = hData.sum('dataset')
hData = hData.sum('category')
hData = hData.sum('systematic')
hData = RebinHist(hData,"mass",5)
hData = SetRangeHist(hData,"mass",40,200)



systematics = h.axis('systematic').identifiers()

for _lepton in ['electron','muon']:
    outputFile = uproot.recreate(f"RootFiles/MisID_Output_{_lepton}.root")

    outputFile["dataObs"] = hist.export1d(hData.integrate("lepFlavor",_lepton))

    hMisID = h.integrate("category","MisIDele").sum("dataset").integrate("lepFlavor",_lepton)
    hOther = h.integrate("category",["Genuine","NonPrompt"]).integrate("lepFlavor",_lepton)
    datasets = hOther.axis('dataset').identifiers()

    for _systematic in systematics:
        outputFile[f'MisIDele_{_systematic}'] = hist.export1d(hMisID.integrate('systematic',_systematic))
        for _dataset in datasets:
            outputFile[f'{_dataset}_{_systematic}'] =  hist.export1d(hOther.integrate("dataset",_dataset).integrate('systematic',_systematic))

    
    outputFile.close()